## Automatizar extracción info PDF desde archivo

### Installing the required libraries

In [1]:
pip install pdfplumber

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import requests
import pdfplumber
import pandas as pd

### Opening de file and extracting info in text

We are using the pdfplumber library that is running OCR pytesseract in the background to parse all the info into text

In [2]:
FILEPATH = '/Users/javiervinas/Documents/Bootcamp_Data/final_project_ironhack/facturas/factura_mayo.pdf'

with pdfplumber.open(FILEPATH) as pdf:
    first_page = pdf.pages[0]
    text = first_page.extract_text()

In [3]:
print(text)

Razón Social: Ana María García Martínez
NIF / CIF: 00654715C
CUPS: ES0022000006297223FV
Dir. Suministro: EMBAJADORES 152, 2º, 6, 28045 MADRID (MADRID)
Contrato Acceso: 001000088135 Tarifa: 2.0DHA
Empresa Distribuidora: UFD DISTRIBUCION ELECTRICIDAD, S.A.
Datos Factura Ana María García Martínez
Fecha de Factura: 25/05/2020 GAZTAMBIDE 70, 3º, C
Periodo Facturación: 20/04/2020 - 20/05/2020 (30 días)
28003 MADRID (MADRID)
Factura Nº: 2020052506693
Nº de Contrato: CO-2018-076542_4.3
Fecha fin del contrato de suministro: 08/11/2020  (renovación anual automática)
Importe Total
Término de energía variable
Precio Peaje Precio Coste Energía Precio Total Consumo Total
P1:  0,062012€/kWh     +           0   , 0  8  7   2 00 €/kWh=         0   , 1  4  9  2   1  2   € /kWh          x                   42,00kWh =                   6   , 2  7    €  8,76 €
P2:  0,002215€/kWh     +           0   , 0  7  5   7 02 €/kWh=         0   , 0  7  7  9   1  7   € /kWh          x                   32,00kWh =     

### Los campos que nos interesaría volcar a nuestra base de datos serían:

- Nombre del cliente
- NIF / CIF
- CUPS
- Dirección suministro
- Contrato acceso
- Tarifa
- Empresa distribuidora
- Fecha factura
- Periodo facturación
- Nº factura
- Nº contrato
- Importe variable
- Término potencia
- Energía reactiva
- Impuesto electricidad
- Alquiler equipo contador
- Importe total sin iva
- IVA
- Total factura

In [7]:
# Nombre del cliente

In [4]:
for row in text.split('\n'):
    if row.startswith('Razón Social:'):
        client_name = row.split()[2:]
        nombre_del_cliente = " ".join(client_name)

In [5]:
nombre_del_cliente

'Ana María García Martínez'

In [6]:
table_info = {'Nombre del cliente': nombre_del_cliente}
print(table_info)

{'Nombre del cliente': 'Ana María García Martínez'}


In [7]:
# NIF / CIF

In [8]:
for row in text.split('\n'):
    if row.startswith('NIF / CIF:'):
        nif_cif = row.split()[-1]

In [9]:
nif_cif

'00654715C'

In [10]:
table_info['NIF / CIF'] = nif_cif

In [11]:
table_info

{'Nombre del cliente': 'Ana María García Martínez', 'NIF / CIF': '00654715C'}

In [12]:
for row in text.split('\n'):
    if row.startswith('TOTAL FACTURA'):
        total_factura = row.split()[-1]

In [13]:
table_info['Total factura'] = total_factura

In [14]:
table_info

{'Nombre del cliente': 'Ana María García Martínez',
 'NIF / CIF': '00654715C',
 'Total factura': '25,80€'}

### Converting all the fields into a pandas dataframe

In [15]:
table_info

df = pd.DataFrame(table_info, index=[1]).T

print(df)

                                            1
Nombre del cliente  Ana María García Martínez
NIF / CIF                           00654715C
Total factura                          25,80€
